# Notebook for generating the FaceNet-based face embeddings from a folder in a given path

In [ ]:
import os
import tensorflow as tf
from align import detect_face
import facenet
import cv2
import imutils
import numpy as np
from tqdm import tqdm_notebook

#important for tf to grab the appropriate ammount of memory and not crash cuda

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

# some image preprocessing constants kept as default from facenet
minsize = 20
threshold = [0.6, 0.7, 0.7]
factor = 0.709
margin = 44
input_image_size = 160

In [ ]:
def getFace(img):
    faces = []
    img_size = np.asarray(img.shape)[0:2]
    bounding_boxes, points = detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
    if not len(bounding_boxes) == 0:
        for face in bounding_boxes:
            if face[4] > 0.50:
                det = np.squeeze(face[0:4])
                bb = np.zeros(4, dtype=np.int32)
                bb[0] = np.maximum(det[0] - margin / 2, 0)
                bb[1] = np.maximum(det[1] - margin / 2, 0)
                bb[2] = np.minimum(det[2] + margin / 2, img_size[1])
                bb[3] = np.minimum(det[3] + margin / 2, img_size[0])
                cropped = img[bb[1]:bb[3], bb[0]:bb[2], :]
                resized = cv2.resize(cropped, (input_image_size,input_image_size),interpolation=cv2.INTER_CUBIC)
                prewhitened = facenet.prewhiten(resized)
                faces.append({'face':resized,'rect':[bb[0],bb[1],bb[2],bb[3]],'embedding':getEmbedding(prewhitened)})
    return faces

def getEmbedding(resized):
    
    reshaped = resized.reshape(-1,input_image_size,input_image_size,3)
    feed_dict = {images_placeholder: reshaped, phase_train_placeholder: False}
    # print(feed_dict)
    embedding = sess.run(embeddings, feed_dict=feed_dict)
    return embedding

In [ ]:
sess = tf.Session()
# read pnet, rnet, onet models from align directory and files are det1.npy, det2.npy, det3.npy
pnet, rnet, onet = detect_face.create_mtcnn(sess, 'align')

# read 20170512-110547 model file downloaded from https://drive.google.com/file/d/0B5MzpY9kBtDVZ2RpVDYwWmxoSUk
facenet.load_model("20170512-110547/20170512-110547.pb")
# facenet.load_model("20180408-102900/20180408-102900.pb") (this is the newer model, produces 512-dim embeddings)

# Get input and output tensors
images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
embedding_size = embeddings.get_shape()[1]

# for general purpose embeddings making

# change selection to the variable of interest; give path

In [ ]:
selection = YourPATH ##### give path to the folder containing your images here #####

imgsNew = [os.path.join(selection, i) for i in os.listdir(selection)]

descs = [i.split('.')[0] for i in os.listdir(selection)]

In [ ]:
bar = tqdm_notebook([(d,i) for d,i in zip(descs, imgsNew)])

embdsComposites = dict()
failedHere = list()

for case, i in bar:
    
    try:
        img = cv2.imread(i)
        img = imutils.resize(img,width=1000)
        faces = getFace(img)

        tmpE = None
    except:
        print('Failed case: {}'.format(case))
        failedHere.append(case)        
        continue

    for face in faces:
                
        tmpE = face['embedding']
        embdsComposites[case] = tmpE
    

# show failed cases

In [ ]:
failedHere

# save embeddings with the appropriate file name; load your dictionary in the analysis notebook

In [ ]:
import pickle

with open('testCases_embeddingsVsComposite', 'wb') as fh:
    
    pickle.dump(embdsComposites, fh)  